# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
F. Walter  ->  F. Walter  |  ['F. Walter']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Zhang  ->  R. Zhang  |  ['Zhang']
T. Henning  ->  T. Henning  |  ['T. Henning']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


J. Henshaw  ->  J. Henshaw  |  ['J. Henshaw']
G.-D. Marleau  ->  G.-D. Marleau  |  ['G.-D. Marleau']
Arxiv has 75 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.11457


extracting tarball to tmp_2409.11457...

 done.
  0: tmp_2409.11457/aassymbols.tex, 579 lines
  1: tmp_2409.11457/main.tex, 772 lines


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2409.11457/main.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2409.12185...

 done.


Found 104 bibliographic references in tmp_2409.12185/aa51190-24corr.bbl.
Retrieving document from  https://arxiv.org/e-print/2409.12187


extracting tarball to tmp_2409.12187... done.


Found 81 bibliographic references in tmp_2409.12187/main.bbl.
syntax error in line 109: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11463-b31b1b.svg)](https://arxiv.org/abs/2409.11463) | **Characterizing the contribution of dust-obscured star formation at $z \gtrsim$ 5 using 18 serendipitously identified [CII] emitters**  |
|| I. F. v. Leeuwen, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-09-19*|
|*Comments*| *Accepted for publication in MNRAS. 24 pages, 14 figures, 8 tables (including appendices)*|
|**Abstract**|            We present a new method to determine the star formation rate (SFR) density of the Universe at $z \gtrsim 5$ that includes the contribution of dust-obscured star formation. For this purpose, we use a [CII] (158 $\mu$m) selected sample of galaxies serendipitously identified in the fields of known $z\gtrsim 4.5$ objects to characterize the fraction of obscured SFR. The advantage of a [CII] selection is that our sample is SFR-selected, in contrast to a UV-selection that would be biased towards unobscured star formation. We obtain a sample of 23 [CII] emitters near star-forming (SF) galaxies and QSOs -- three of which we identify for the first time -- using previous literature and archival ALMA data. 18 of these serendipitously identified galaxies have sufficiently deep rest-UV data and are used to characterize the obscured fraction of the star formation in galaxies with SFRs $\gtrsim 30\ \text{M}_{\odot} \ \text{yr}^{-1}$. We find that [CII] emitters identified around SF galaxies have $\approx$63\% of their SFR obscured, while [CII] emitters around QSOs have $\approx$93\% of their SFR obscured. By forward modeling existing wide-area UV luminosity function (LF) determinations, we derive the intrinsic UV LF using our characterization of the obscured SFR. Integrating the intrinsic LF to $M_{UV}$ = $-$20 we find that the obscured SFRD contributes to $>3\%$ and $>10\%$ of the total SFRD at $z \sim 5$ and $z \sim 6$ based on our sample of companions galaxies near SFGs and QSOs, respectively. Our results suggest that dust obscuration is not negligible at $z\gtrsim 5$, further underlining the importance of far-IR observations of the $z\gtrsim 5$ Universe.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11485-b31b1b.svg)](https://arxiv.org/abs/2409.11485) | **Survey of Orion Disks with ALMA (SODA) III: Disks in wide binary systems in L1641 and L1647**  |
|| G. Ricciardi, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2024-09-19*|
|*Comments*| *18 pages, 30 figures, comments welcome*|
|**Abstract**|            Aims. The goal of this work is to comprehensively characterize the impact of stellar multiplicity on Class II disks in the L1641 and L1647 regions of Orion A (~1-3 Myr), part of the Survey of Orion Disks with ALMA (SODA). We characterize the protostellar multiplicity using the Atacama Large Millimeter/submillimeter Array (ALMA), the ESO-VISTA, and Hubble Space telescopes. The resulting sample of 65 multiple systems represents the largest catalogue of wide binary systems to date (projected separation >1000 AU), allowing a more robust statistical characterization of the evolution and properties of protoplanetary disks. Methods. The disk population was observed in continuum with ALMA at 225 GHz, with a median rms of 1.5 Mearth. Combining these data (resolution ~1.1arcsec ) with the ESO-VISTA near-infrared survey of the Orion A cloud (resolution ~0.7arcsec ), multiple systems are assembled and selected by an iterative inside-out search in projected separation (>1000 AU). Results. We identify 61 binary systems, 3 triple systems, and one quadruple system. The separation range is between 1000 and 10^4 AU. The dust mass distributions inferred with the Kaplan-Meier estimator yield a median mass of 3.23+0.6-0.4 Mearth for primary disks and 3.88+0.3-0.3 Mearth for secondary disks.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.12185-b31b1b.svg)](https://arxiv.org/abs/2409.12185) | **Disruption of a massive molecular cloud by a supernova in the Galactic Centre: Initial results from the ACES project**  |
|| M. Nonhebel, et al. -- incl., <mark>J. Henshaw</mark> |
|*Appeared on*| *2024-09-19*|
|*Comments*| *17 pages, 13 figures, and 2 tables. Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            The Milky Way's Central Molecular Zone (CMZ) differs dramatically from our local solar neighbourhood, both in the extreme interstellar medium conditions it exhibits (e.g. high gas, stellar, and feedback density) and in the strong dynamics at play (e.g. due to shear and gas influx along the bar). Consequently, it is likely that there are large-scale physical structures within the CMZ that cannot form elsewhere in the Milky Way. In this paper, we present new results from the Atacama Large Millimeter/submillimeter Array (ALMA) large programme ACES (ALMA CMZ Exploration Survey) and conduct a multi-wavelength and kinematic analysis to determine the origin of the M0.8$-$0.2 ring, a molecular cloud with a distinct ring-like morphology. We estimate the projected inner and outer radii of the M0.8$-$0.2 ring to be 79" and 154", respectively (3.1 pc and 6.1 pc at an assumed Galactic Centre distance of 8.2 kpc) and calculate a mean gas density $> 10^{4}$ cm$^{-3}$, a mass of $\sim$ $10^6$ M$_\odot$, and an expansion speed of $\sim$ 20 km s$^{-1}$, resulting in a high estimated kinetic energy ($> 10^{51}$ erg) and momentum ($> 10^7$ M$_\odot$ km s$^{-1}$). We discuss several possible causes for the existence and expansion of the structure, including stellar feedback and large-scale dynamics. We propose that the most likely cause of the M0.8$-$0.2 ring is a single high-energy hypernova explosion. To viably explain the observed morphology and kinematics, such an explosion would need to have taken place inside a dense, very massive molecular cloud, the remnants of which we now see as the M0.8$-$0.2 ring. In this case, the structure provides an extreme example of how supernovae can affect molecular clouds.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.12187-b31b1b.svg)](https://arxiv.org/abs/2409.12187) | **Exoplanet accretion monitoring spectroscopic survey (ENTROPY) I. Evidence for magnetospheric accretion in the young isolated planetary-mass object 2MASS J11151597+1937266**  |
|| G. Viswanath, et al. -- incl., <mark>G.-D. Marleau</mark> |
|*Appeared on*| *2024-09-19*|
|*Comments*| *Accepted for publication at A&A on 2024-09-11. 15 pages, 9 figures, 7 tables*|
|**Abstract**|            Accretion among planets is a poorly understood phenomenon, due to lack of both observational and theoretical studies. Detection of emission lines from accreting gas giants facilitate detailed investigations into this process. This work presents a detailed analysis of Balmer lines from one of the few known young, planetary-mass objects with observed emission, the isolated L2 dwarf 2MASS J11151597+1937266 with a mass 7-21 Mj and age 5-45 Myr, located at 45+-2 pc. We obtained the first high-resolution (R~50,000) spectrum of the target with VLT/UVES, a spectrograph in the near-UV to visible wavelengths (3200-6800 AA). We report resolved H3-H6 and He I (5875.6 AA) emission in the spectrum. Based on the asymmetric line profiles of H3 and H4, 10% width of H3 (199+-1 km/s), tentative He I 6678 AA emission and indications of a disk from MIR excess, we confirm ongoing accretion at this object. Using the Gaia update of the parallax, we revise its temperature to 1816+-63 K and radius to 1.5+-0.1 Rj. Analysis of observed H I profiles using 1D planet-surface shock model implies a pre-shock gas velocity of v0=120(+80,-40) km/s and a pre-shock density of log(n0/cm^-3)=14(+0,-5). Pre-shock velocity points to a mass of 6(+8,-4) Mj for the target. Combining the H I line luminosities and planetary Lline-Lacc scaling relations, we derive a mass accretion rate of 1.4(+2.8,-0.9)x10^-8 Mj/yr.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11481-b31b1b.svg)](https://arxiv.org/abs/2409.11481) | **The MDW H{\alpha} Sky Survey: Data Release 0**  |
|| N. Aftab, et al. -- incl., <mark>Zhang</mark> |
|*Appeared on*| *2024-09-19*|
|*Comments*| **|
|**Abstract**|            The Mittelman-di Cicco-Walker (MDW) H$\alpha$ Sky Survey is an autonomously-operated and ongoing all-sky imaging survey in the narrowband H$\alpha$ wavelength. The survey was founded by amateur astronomers, and is presented here in its first stage of refinement for rigorous scientific use. Each field is exposed through an H$\alpha$ filter with a 3nm bandwidth for a total of four hours, with a pixel scale of 3.2 arcsec. Here, we introduce the first Data Release of the MDW H$\alpha$ Survey (Data Release 0, or DR0), spanning 238 fields in the region of Orion (~3100 deg$^2$). DR0 includes: calibrated mean fields, star-removed mean fields, a point source catalog matched to Data Release 1 of the Panoramic Survey Telescope and Rapid Response System (Pan-STARRS1) and the INT Galactic Plane Survey (IGAPS), and mosaics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11767-b31b1b.svg)](https://arxiv.org/abs/2409.11767) | **Bright unintended electromagnetic radiation from second-generation Starlink satellites**  |
|| C. Bassa, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-09-19*|
|*Comments*| *12 pages, 6 figures, accepted for publication in A&A*|
|**Abstract**|            We report on the detection of unintended electromagnetic radiation (UEMR) from the second-generation of Starlink satellites. Observations with the LOFAR radio telescope between 10 to 88MHz and 110 to 188MHz show broadband emission covering the frequency ranges from 40 to 70MHz and 110 to 188MHz from the v2-Mini and v2-Mini Direct-to-Cell Starlink satellites. The spectral power flux density of this broadband UEMR varies from satellite to satellite, with values ranging from 15Jy to 1300Jy, between 56 and 66MHz, and from 2 to 100Jy over two distinct 8MHz frequency ranges centered at 120 and 161MHz. We compared the detected power flux densities of this UEMR to that emitted by the first generation v1.0 and v1.5 Starlink satellites. When correcting for the observed satellite distances, we find that the second-generation satellites emit UEMR that is up to a factor of 32 stronger compared to the first generation. The calculated electric field strengths of the detected UEMR exceed typical electromagnetic compatibility standards used for commercial electronic devices as well as recommended emission thresholds from the Radiocommunication Sector of the International Telecommunications Union (ITU-R) aimed at protecting the 150.05-153MHz frequency range allocated to radio astronomy. We characterize the properties of the detected UEMR with the aim of assisting the satellite operator with the identification of the cause of the UEMR.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11457-b31b1b.svg)](https://arxiv.org/abs/2409.11457) | **UNCOVER: Significant Reddening in Cosmic Noon Quiescent Galaxies**  |
|| J. Siegel, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-09-19*|
|*Comments*| *18 pages, 8 figures, submitted to ApJ*|
|**Abstract**|            We explore the physical properties of five massive quiescent galaxies at $z\sim2.5$, revealing the presence of non-negligible dust reservoirs. JWST NIRSpec observations were obtained for each target, finding no significant line emission; multiple star formation tracers independently place upper limits between $0.1-10~M_\odot / \mathrm{yr}$. Spectral energy distribution modeling with Prospector infers stellar masses between $\log_{10}[M / M_\odot] \sim 10-11$ and stellar mass-weighted ages between $1-2$ Gyr. The inferred mass-weighted effective radii ($r_{eff}\sim 0.4-1.4$ kpc) and inner $1$ kpc stellar surface densities ($\log_{10}[\Sigma / M_\odot \mathrm{kpc}^2 ]\gtrsim 9$) are typical of quiescent galaxies at $z \gtrsim 2$. The galaxies display negative color gradients (redder core and bluer outskirts); for one galaxy, this effect results from a dusty core, while for the others it may be evidence of an "inside-out" growth process. Unlike local quiescent galaxies, we identify significant reddening in these typical cosmic noon passive galaxies; all but one require $A_V \gtrsim 0.4$. This finding is in qualitative agreement with previous studies but our deep 20-band NIRCam imaging is able to significantly suppress the dust-age degeneracy and confidently determine that these galaxies are reddened. We speculate about the physical effects that may drive the decline in dust content in quiescent galaxies over cosmic time.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 36</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11464-b31b1b.svg)](https://arxiv.org/abs/2409.11464) | **A Systematic Search for Galaxies with Extended Emission Line and Potential Outflows in JADES Medium-Band Images**  |
|| Y. Zhu, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-09-19*|
|*Comments*| *21 pages, 12 figures, 2 tables. Submitted to AAS journals*|
|**Abstract**|            For the first time, we systematically search for galaxies with extended emission line and potential outflows features using medium-band images in the GOODS-S field by comparing the morphology in medium-band images to adjacent continuum and UV bands. We look for galaxies that have a maximum extent 50\% larger, an excess area 30\% greater, or an axis ratio difference of more than 0.3 in the medium band compared to the reference bands. After visual inspection, we find 326 candidate galaxies at $1 < z < 6$, with a peak in the population near cosmic noon, benefiting from the good coverage of the medium-band filters. By examining their SEDs, we find that the candidate galaxies are at least 20\% more bursty in their star-forming activity and have 60\% more young stellar populations compared to a control sample selected based on the continuum band flux. Additionally, these candidates exhibit a significantly higher production rate of ionizing photons. We further find that candidates hosting known AGN produce extended emission that is more anisotropic compared to non-AGN candidates. A few of our candidates have been spectroscopically confirmed to have prominent outflow signatures through NIRSpec observations, showcasing the robustness of the photometric selection. Future spectroscopic follow-up will better help verify and characterize the kinematics and chemical properties of these systems.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.11463/./Figures/SFR_obs_SFR_tot.png', 'tmp_2409.11463/./Figures/SFRD_2_panels.png', 'tmp_2409.11463/./Figures/parameter_space.png']
copying  tmp_2409.11463/./Figures/SFR_obs_SFR_tot.png to _build/html/
copying  tmp_2409.11463/./Figures/SFRD_2_panels.png to _build/html/
copying  tmp_2409.11463/./Figures/parameter_space.png to _build/html/
exported in  _build/html/2409.11463.md
    + _build/html/tmp_2409.11463/./Figures/SFR_obs_SFR_tot.png
    + _build/html/tmp_2409.11463/./Figures/SFRD_2_panels.png
    + _build/html/tmp_2409.11463/./Figures/parameter_space.png
found figures ['tmp_2409.11485/./Flux_Separation_Mass1.png', 'tmp_2409.11485/./Bestfit_linearconst_up.png', 'tmp_2409.11485/./Multiples_Comparison.png']
copying  tmp_2409.11485/./Flux_Separation_Mass1.png to _build/html/
copying  tmp_2409.11485/./Bestfit_linearconst_up.png to _build/html/
copying  tmp_2409.11485/./Multiples_Comparison.png to _build/html/
exported in  _build/html/2409.11485.md
    + _build/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Characterizing the contribution of dust-obscured star formation at $z \gtrsim$ 5 using18 serendipitously identified [CII] emitters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.11463-b31b1b.svg)](https://arxiv.org/abs/2409.11463)<mark>Appeared on: 2024-09-19</mark> -  _Accepted for publication in MNRAS. 24 pages, 14 figures, 8 tables (including appendices)_

</div>
<div id="authors">

I. F. v. Leeuwen, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ We present a new method to determine the star formation rate (SFR) density of the Universe at $z \gtrsim 5$ that includes the contribution of dust-obscured star formation. For this purpose, we use a [ CII ] (158 $\mu$ m) selected sample of galaxies serendipitously identified in the fields of known $z\gtrsim 4.5$ objects to characterize the fraction of obscured SFR. The advantage of a [ CII ] selection is that our sample is SFR-selected, in contrast to a UV-selection that would be biased towards unobscured star formation. We obtain a sample of 23 [ CII ] emitters near star-forming (SF) galaxies and QSOs -- three of which we identify for the first time -- using previous literature and archival ALMA data. 18 of these serendipitously identified galaxies have sufficiently deep rest-UV data and are used to characterize the obscured fraction of the star formation in galaxies with SFRs $\gtrsim 30 \text{M}_{\odot}   \text{yr}^{-1}$ . We find that [ CII ] emitters identified around SF galaxies have $\approx$ 63 \% of their SFR obscured, while [ CII ] emitters around QSOs have $\approx$ 93 \% of their SFR obscured. By forward modeling existing wide-area UV luminosity function (LF) determinations, we derive the intrinsic UV LF using our characterization of the obscured SFR. Integrating the intrinsic LF to $M_{UV}$ = $-$ 20 we find that the obscured SFRD contributes to $>3\%$ and $>10\%$ of the total SFRD at $z \sim 5$ and $z \sim 6$ based on our sample of companions galaxies near SFGs and QSOs, respectively. Our results suggest that dust obscuration is not negligible at $z\gtrsim 5$ , further underlining the importance of far-IR observations of the $z\gtrsim 5$ Universe.

</div>

<div id="div_fig1">

<img src="tmp_2409.11463/./Figures/SFR_obs_SFR_tot.png" alt="Fig8" width="100%"/>

**Figure 8. -** The unobscured fraction of the SFR as a function of the total SFR as measured from the [CII] emission from companions galaxies around SF galaxies (_left_) and companions galaxies around QSOs (_right_). The [CII]-selected galaxies are shown in red and orange for these samples, respectively, and the mean unobscured fraction by the black solid line. The width of the shaded grey region indicates the 16th and 84th percentiles of the mean fraction. Galaxies that are not detected in the far-IR are indicated by an upside down triangle. The blue diamonds show the relation derived for the stacked UV-selected ALPINE galaxies \citep{bethermin2020}, while the purple crosses show the relation based on the stacked REBELS sample \citep{algera23}. The dotted lines show the $z \sim 5$ and $z \sim 6$ relations from \citet{mauerhofer} from the DELPHI semi-analytic model calibrated with ALPINE and REBELS galaxies. Also shown are the IRX-$M_{*}$  relations from ASPECS \citep{Bouwens2020},  A$^3$COSMOS \citep{fudamoto20} and ALPINE/REBELS \citep{bowler23} assuming the main sequence relation from \citet{speagle14} for $z = 5$ and $z = 6$. The top axis shows the stellar masses assuming the same relation from \citet{speagle14} at $z = 5$ and $z = 6$. (*fig:obs*)

</div>
<div id="div_fig2">

<img src="tmp_2409.11463/./Figures/SFRD_2_panels.png" alt="Fig11" width="100%"/>

**Figure 11. -** Constraints on the SFR density as a function of redshift. _Top panel:_ The red and orange solid circles show the SFR density we estimate from our intrinsic UV luminosity functions based on dust obscuration from companions around SF galaxies and QSOs, respectively, integrated to $-17$ mag. We only infer the intrinsic LF and dust corrections over the SFR range directly sampled by our work and therefore the dust corrected SFRDs we derive are lower limits. The blue solid circles show the SFR density when we integrate the observed UV LF from \citet{Harikane2022} to $-17$ mag. The solid black line indicates the fiducial SFR density evolution from \citet{Madau2014}.  Meanwhile, the dashed/dotted lines correspond to the dust-rich and dust-poor models from \citet{casey2018brightest} and intrinsic and dust-attenuated models from \citet{mauerhofer}. Furthermore, we show the constraints on the rest-UV SFRD from \citet{bouwens2022}(_grey triangles_), the SFRD from radio-observations from \citet{Vlugt22}(_green triangles_) and SFRDs from the [CII]-LF from the ALPINE companions from \citet{Loiacona_alpine}(_aqua and purple crosses_).
    _Lower panel:_ The purple and green solid circles show the contribution of dust-obscured sources to the SFR density, based on the dust obscuration from SF galaxy and QSO companions, respectively. The empty purple and green circles show the dust-obscured SFRD when we assume a constant mean obscuration instead of using the distribution of obscured fractions. Other estimates of the contribution of obscured sources to the SFRD are shown, including those of \citet{Casey2021_MORA}(_light blue triangles_), \citet{Wang2019}(_orange pentagons_), \citet{Barrufet}(_gray thin diamond_), \citet{algera}(_yellow triangle_), \citet{fudamoto_nature}(_pink star_),  \citet{gruppioni2020}(_pink diamonds_),  \citet{Traina}(_aqua squares_), \citet{Khusanova_21}(_red cross and plus sign_), \citet{dud+20}(_green shaded region_) and \citet{vlugt23}(_orange shaded region_). (*fig:SFRD*)

</div>
<div id="div_fig3">

<img src="tmp_2409.11463/./Figures/parameter_space.png" alt="Fig1" width="100%"/>

**Figure 1. -** The redshifts and [CII] luminosities of the 161 ALMA targets we use to search for a sample of [CII]-emitting companion galaxies.  The [CII] luminosity of the archival sources is derived based on the peak flux and the redshift is from the peak frequency of [CII]. The red circles correspond to the 27 quasars with [CII] detections where \citet{venemans_kilo} has already described a companion search, while the purple stars and blue diamonds correspond to the 25 [CII]-detected galaxies from REBELS \citep{rebels} and 75 [CII]-detected galaxies from ALPINE \citep{alpine}, where companion searches have been done (\citealt{fudamoto_nature,Loiacona_alpine}; see also Schouws et al. 2024, in prep.).
    Green squares indicate the 34 [CII]-detected ALMA targets where no searches for [CII]-emitting companion galaxies have thus far been reported in the literature, and we provide for the first time here.
    Filled markers indicate targeted galaxies with serendipitously detected companions, while open markers are targeted galaxies without detected companions. (*fig:parameter_space*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.11463"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Survey of Orion Disks with ALMA (SODA)

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.11485-b31b1b.svg)](https://arxiv.org/abs/2409.11485)<mark>Appeared on: 2024-09-19</mark> -  _18 pages, 30 figures, comments welcome_

</div>
<div id="authors">

G. Ricciardi, et al. -- incl., <mark>S. v. Terwisga</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Observations of protoplanetary disks within multiple systems in nearby star-forming regions (SFRs) have shown that the presence of a neighboring object influences the evolution of dust of disks. However, the sample size available and the separation range covered are insufficient to fully understand the dust evolution in binary systems. The goal of this work is to comprehensively characterize the impact of stellar multiplicity on Class II disks in the L1641 and L1647 regions of Orion A ( $\sim$ 1-3 Myr), part of the Survey of Orion Disks with ALMA (SODA). We characterize the protostellar multiplicity using the Atacama Large Millimeter/submillimeter Array (ALMA), the ESO-VISTA, and Hubble Space telescopes. The resulting sample of 65 multiple systems represents the largest catalogue of wide binary systems to date (projected separation $\geq$ 1000 AU), allowing a more robust statistical characterization of the evolution and properties of protoplanetary disks. The disk population was observed in continuum with ALMA at 225 GHz, with a median rms of 1.5 M $_{\oplus}$ . Combining these data (resolution $\sim$ 1.1 $^{"}$ ) with the ESO-VISTA near-infrared survey of the Orion A cloud (resolution $\sim$ 0.7 $^{"}$ ), multiple systems are assembled and selected by an iterative inside-out search in projected separation ( $\geq$ 1000 AU). We identify 61 binary systems, 3 triple systems, and one quadruple system. The separation range is between 1000 and 10 $^4$ AU. The dust mass distributions inferred with the Kaplan-Meier estimator yield a median mass of 3.23 $^{+0.6}_{-0.4}$ M $_{\oplus}$ for primary disks and 3.88 $^{+0.3}_{-0.3}$ M $_{\oplus}$ for secondary disks. Combining our data with those available for Lupus and Taurus disks, we identify a threshold separation of about 130 AU, beyond which the previously observed positive correlation between millimeter flux (and hence dust mass) and projected separation is lost. Recent theoretical models confirm that pre- and post-threshold systems are the result of different star formation processes, such as the fragmentation of a gravitationally unstable circumstellar disks, the thermal fragmentation of infalling cores, or the turbulent fragmentation of molecular clouds. We can rule out the dependency on different SFRs: the cumulative mass distribution of multiples in SFRs of similar age are statistically indistinguishable. This result strengthens the hypothesis that there is a universal initial mass distribution for disks.

</div>

<div id="div_fig1">

<img src="tmp_2409.11485/./Flux_Separation_Mass1.png" alt="Fig30" width="100%"/>

**Figure 30. -** Pair millimeter flux (re-scaled to a distance of $d=140 pc$) and dust mass of binary systems as a function of their projected separation, located in Orion A, Lupus  ([Zurlo, Cieza and Ansdell 2021](), [Ansdell, Williams and Trapman 2018]()) , and Taurus  ([Akeson, Jensen and Carpenter 2020](), [Manara, Tazzari and Long 2019](), [ and Akeson 2014](),  Harris_2012) . Crosses represent detections and triangles represent non-detections: according to [Akeson, Jensen and Carpenter (2020)](), we consider a binary pair to be detected only if both disk components were detected. In blue is the linear regression (slope $m=0.69^{+0.13}_{-0.12}$, intercept $q=-0.45^{+0.31}_{-0.33}$) from [Zagaria, Rosotti and Lodato (2021)](), and in purple is the linear regression (slope $m=0.15^{+0.11}_{-0.12}$, intercept $q=0.92^{+0.34}_{-0.32}$) we get considering also our binary sample. (*figure:fluxsep*)

</div>
<div id="div_fig2">

<img src="tmp_2409.11485/./Bestfit_linearconst_up.png" alt="Fig31" width="100%"/>

**Figure 31. -** Regression best-fit of the logarithm of flux over the logarithm of separation. Free parameters: m (slope),  q (intercept), and x$_{edge}$(i.e., separation threshold).\ Crosses indicate detected systems, while triangles indicate upper limits. The binary systems are color-coded as in Figure \ref{figure:fluxsep}. (*figure:linearconst*)

</div>
<div id="div_fig3">

<img src="tmp_2409.11485/./Multiples_Comparison.png" alt="Fig1" width="100%"/>

**Figure 1. -** Dust mass distributions inferred with the Kaplan-Meier estimator for the binary systems located in Orion A, Lupus, Taurus, and Upper Scorpius. For the later 3 regions, we have considered the available data from the relevant literature  ([Zurlo, Cieza and Ansdell 2021](), [Ansdell, Williams and Trapman 2018](), [Akeson, Jensen and Carpenter 2020](), [Manara, Tazzari and Long 2019](), [ and Akeson 2014](),  Harris_2012,  Barenfeld_2019) . (*figure:literature*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.11485"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\HII}{\ion{H}{II}}$
$\newcommand{\Ha}{H{\alpha}}$
$\newcommand{\kms}{\rm km s^{-1}}$
$\newcommand{\xx}[1]{\textcolor{black}{ #1}}$
$\newcommand\kms{{\rm km s^{-1}}}$
$\newcommand\Kkms{{\rm K km s^{-1}}}$
$\newcommand\msun{{\mathrm{M}_\odot}}$
$\newcommand\lsun{{\mathrm{L}_\odot}}$
$\newcommand\micron{\hbox{\mum}}$
$\newcommand\solyr{{\rm M_\odot yr^{-1}}}$
$\newcommand\sol{{\rm M_\odot}}$
$\newcommand\dyncm{{\rm dyn cm^{-2}}}$
$\newcommand\ergs{{\rm  erg s^{-1}}}$
$\newcommand\cmsq{{\rm cm^{-2}}}$
$\newcommand\cmcb{{\rm cm^{-3}}}$
$\newcommand\gcmsq{{\rm g cm^{-2}}}$
$\newcommand\gcmcb{{\rm g cm^{-3}}}$
$\newcommand\Kcmcb{{\rm K cm^{-3}}}$
$\newcommand\ntdp{{\rm N_2D^+}}$
$\newcommand\name{{\rm the vortex}}$</div>



<div id="title">

# Disruption of a massive molecular cloud by a supernova in the Galactic Centre

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.12185-b31b1b.svg)](https://arxiv.org/abs/2409.12185)<mark>Appeared on: 2024-09-19</mark> -  _17 pages, 13 figures, and 2 tables. Accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

M. Nonhebel, et al.

</div>
<div id="abstract">

**Abstract:** The Milky Way's Central Molecular Zone (CMZ) differs dramatically from our local solar neighbourhood, both in the extreme interstellar medium conditions it exhibits (e.g. high gas, stellar, and feedback density) and in the strong dynamics at play (e.g. due to shear and gas influx along the bar).Consequently, it is likely that there are large-scale physical structures within the CMZ that cannot form elsewhere in the Milky Way.In this paper, we present new results from the Atacama Large Millimeter/submillimeter Array (ALMA) large programme ACES (ALMA CMZ Exploration Survey) and conduct a multi-wavelength and kinematic analysis to determine the origin of the M0.8 $-$ 0.2 ring, a molecular cloud with a distinct ring-like morphology. We estimate the projected inner and outer radii of the M0.8 $-$ 0.2 ring to be 79 $\arcsec$ and 154 $\arcsec$ , respectively (3.1 pc and 6.1 pc at an assumed Galactic Centre distance of 8.2 kpc) and calculate a mean gas density $> 10^{4}$ cm $^{-3}$ , a mass of $\sim$ $10^6$ $\sol$ , and an expansion speed of $\sim$ 20 $\kms$ , resulting in a high estimated kinetic energy ( $> 10^{51}$ erg) and momentum ( $> 10^7$ $\textup{M}_{\odot}$ $\kms$ ). We discuss several possible causes for the existence and expansion of the structure, including stellar feedback and large-scale dynamics. We propose that the most likely cause of the M0.8 $-$ 0.2 ring is a single high-energy hypernova explosion. To viably explain the observed morphology and kinematics, such an explosion would need to have taken place inside a dense, very massive molecular cloud, the remnants of which we now see as the M0.8 $-$ 0.2 ring. In this case, the structure provides an extreme example of how supernovae can affect molecular clouds.

</div>

<div id="div_fig1">

<img src="tmp_2409.12185/./figures/multiwave_all_max_labels_updated_v4.png" alt="Fig7" width="100%"/>

**Figure 7. -** {Multi-wavelength view of the M0.8$-$0.2 ring.}
    The dotted white circles in each panel show the approximate extent of the _Herschel_ 250 $\mu$m continuum emission of the ring. The cyan circle marks the position of the $\ion${H}{ii} region discussed in Sect. \ref{subsec_youngfeedback}.
    _Top row, left to right:__Chandra_ 4 - 6 keV emission  ([ and Wang 2021]()) , 2MASS K-band observations  ([Skrutskie, Cutri and Stiening 2006]()) , and _Spitzer_ 8 $\mu$m observations from the GLIMPSE survey  ([Churchwell, Babler and Meade 2009]()) . _Middle row, left to right:__Spitzer_ MIPSGAL 24 $\mu$m  ([Carey, Noriega-Crespo and Mizuno 2009]())  and _Herschel_  70 and 250 $\mu$m observations  ([Molinari, Swinyard and Bally 2010]()) . _Bottom row, left to right:_ 3 mm continuum emission from the ACES survey (12 m combined with GBT+_Planck_ from \citealp{Ginsburg2020}), 20 cm emission observed by MeerKAT, and the MeerKAT spectral index map, with a zoomed-in view of the peak H40$\alpha$ emission at the location of the $\ion${H}{ii} region overlaid.
    In the 3 mm continuum image, magnetic field pseudo-vectors from the FIREPLACE polarimetry survey are overlaid (polarisation vectors rotated by 90 degrees, as the direction of the polarised electric field measured in the infrared is considered to be perpendicular to that of the magnetic field; \citealp{Butterfield2023, Butterfield2024}).
    The beam sizes of the observations are shown with a circle in the lower-right corner of each panel, with the FWHMs as follows (proceeding from left to right and top to bottom, and finishing with the H40$\alpha$ image): 0.5, 4, 2, 6, 5, 18, 2.5, 4, 4, and 1.4$\arcsec$. A scale bar of 5 pc is displayed in yellow on the _Chandra_ image. (*fig_multiwavelength*)

</div>
<div id="div_fig2">

<img src="tmp_2409.12185/./figures/hnco_rgb_v3.png" alt="Fig5" width="100%"/>

**Figure 5. -** {Overview of the M0.8$-$0.2 ring within the CMZ.}_Upper-left panel:_ HNCO (4--3) peak intensity map (12 m, 7 m, and total power array data combined) from ACES towards the M0.8$-$0.2 ring. The colour scale extends from 0.2 to 8.0 K in square root stretch. The dotted white circles show the approximate extent of the _Herschel_ 250 $\mu$m continuum emission of the ring-like structure. The ALMA beam size of 2.8$\arcsec$ is shown in the lower right, and a scale bar of 5 pc is shown in the lower left of the panel. _Lower-left panel:_ HNCO (4--3) emission from the Mopra telescope  ([Jones, Burton and Cunningham 2012]()) . The Mopra beam size of 39$\arcsec$ is shown in the lower right of the panel. _Upper-right panel:_ Multi-wavelength view of part of the CMZ. We show a three-colour composite of 8 \micron emission  from the * Spitzer* GLIMPSE survey \citep[green;][]{Churchwell2009}, 24 \micron emission from the * Spitzer* MIPSGAL survey \citep[yellow;][]{Carey2009}, and 20 cm emission observed by MeerKAT \citep[red;][]{Heywood2019, Heywood2022} and the GBT (red; \citealp{Law2008}). Overlaid as black contours are the total molecular gas column densities at levels of 5, 7.5, 10, 15, 25, and 50 $\times  10^{22}$ cm$^{-2}$(Battersby et al., in prep) calculated using data from the * Herschel* Hi-GAL survey  ([Molinari, Swinyard and Bally 2010]()) . Also overlaid are a number of interesting features in the Galactic Centre and the dotted white circles highlighting the ring-like structure (identical to the left panels). The _Herschel_ beam size of 25$\arcsec$ and a scale bar of 15 pc are shown in the lower left of the panel. _Lower-right panel:_ PV diagram of the Mopra HNCO (4--3) emission  ([Henshaw, Longmore and Kruijssen 2016]()) . Each point corresponds to the longitude and centroid velocity of a Gaussian component of the HNCO emission, extracted using {\sc scouse} ([Henshaw, Longmore and Kruijssen 2016]()) . The emission at the position of the ring-like structure is highlighted with three different colours, corresponding to the velocity ranges <0 $\kms$(blue), 0--70 $\kms$(orange), and >70 $\kms$(red). The orange dots correspond to the M0.8$-$0.2 ring, while the blue and red dots relate to additional line-of-sight material. The orbital model of [Kruijssen, Dale and Longmore (2015)]() is overlaid as a dashed black line. (*fig_CMZ_HNCO_peak*)

</div>
<div id="div_fig3">

<img src="tmp_2409.12185/./figures/radial-profile.png" alt="Fig1" width="100%"/>

**Figure 1. -** {Radial profile from _Herschel_ 250 $\mu$m emission.} The mean flux contained in annuli of increasing radius is plotted, normalised against the peak value. The annuli are defined outwards from the centre of the M0.8-0.2 ring ($l=0.^{\circ}804$ and $b=-0.^{\circ}184$) with a thickness of one pixel. The overlaid vertical grey lines define the radii between which the normalised mean flux is above the threshold of 0.5, indicated by the horizontal grey line. (*fig_chans*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.12185"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mj}{\ensuremath{M_\mathrm{Jup}}\xspace}$
$\newcommand{\rj}{\ensuremath{R_\mathrm{Jup}}\xspace}$
$\newcommand{\mjyr}{\ensuremath{M_\mathrm{Jup} \mathrm{yr^{-1}}}\xspace}$
$\newcommand{\Msyr}{\ensuremath{M_\odot \mathrm{yr^{-1}}}\xspace}$
$\newcommand{\lsun}{\ensuremath{L_\odot}\xspace}$
$\newcommand{\msun}{\ensuremath{M_\odot}\xspace}$
$\newcommand{\rsun}{\ensuremath{R_\odot}\xspace}$
$\newcommand{\Mp}{\ensuremath{M_{\mathrm{p}}}\xspace}$
$\newcommand{\Rp}{\ensuremath{R_{\mathrm{p}}}\xspace}$
$\newcommand{\Mstar}{\ensuremath{M_\star}\xspace}$
$\newcommand{\Rstar}{\ensuremath{R_\star}\xspace}$
$\newcommand{\Teff}{\ensuremath{T_{\mathrm{eff}}}\xspace}$
$\newcommand{\Ha}{\ensuremath{\mathrm{H}\alpha}\xspace}$
$\newcommand{\Hb}{\ensuremath{\mathrm{H}\beta}\xspace}$
$\newcommand{\Hg}{\ensuremath{\mathrm{H}\gamma}\xspace}$
$\newcommand{\Lbol}{\ensuremath{L_{\mathrm{bol}}}\xspace}$
$\newcommand{\Lacc}{\ensuremath{L_{\mathrm{acc}}}\xspace}$
$\newcommand{\mdot}{\ensuremath{\dot{M}_{\mathrm{acc}}}\xspace}$
$\newcommand{\logm}{\ensuremath{\mathrm{log}(\mdot/\mjyr)}\xspace}$
$\newcommand{\Lline}{\ensuremath{L_{\mathrm{line}}}\xspace}$
$\newcommand{\logla}{\ensuremath{\mathrm{log}(\Lacc/\lsun)}\xspace}$
$\newcommand{\logll}{\ensuremath{\mathrm{log}(\Lline/\lsun)}\xspace}$
$\newcommand{\Hi}{H~\textsc{i}\xspace}$
$\newcommand{\Hei}{He~\textsc{i}\xspace}$
$\newcommand{\ff}{\ensuremath{f_{f}}\xspace}$
$\newcommand{\lamrest}{\ensuremath{\lambda_\mathrm{rest, air}}\xspace}$
$\newcommand{\lamobs}{\ensuremath{\lambda_\mathrm{observed}}\xspace}$
$\newcommand{\Wten}{\ensuremath{W_{10}}\xspace}$
$\newcommand{\kms}{\ensuremath{\mathrm{km s}^{-1}}\xspace}$
$\newcommand{\flux}{\ensuremath{\mathrm{erg s^{-1} cm^{-2} Å^{-1}}}\xspace}$
$\newcommand{\fluxinteg}{\ensuremath{\mathrm{erg s^{-1} cm^{-2}}}\xspace}$
$\newcommand{\masyr}{\ensuremath{\mathrm{mas yr^{-1}}}\xspace}$
$\newcommand{\cmcube}{\ensuremath{\mathrm{cm}^{-3}}\xspace}$
$\newcommand{\col}{\textcolor{magenta}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Exoplanet accretion monitoring spectroscopic survey (ENTROPY)

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.12187-b31b1b.svg)](https://arxiv.org/abs/2409.12187)<mark>Appeared on: 2024-09-19</mark> -  _Accepted for publication at A&A on 2024-09-11. 15 pages, 9 figures, 7 tables_

</div>
<div id="authors">

G. Viswanath, et al. -- incl., <mark>G.-D. Marleau</mark>

</div>
<div id="abstract">

**Abstract:** Accretion among planetary mass companions is a poorly understood phenomenon, due to the lack of both observational and theoretical studies. Detection of emission lines from accreting gas giants facilitate detailed investigations into this process. This work presents a detailed analysis of Balmer lines from one of the few known young, planetary-mass objects with observed emission, the isolated L2 $\gamma$ dwarf 2MASS J11151597+1937266 with a mass between 7 and 21 $\mj$ and an age of 5--45 Myr, located at $45\pm2$ pc. We obtained the first high-resolution ( $R\sim50,000$ ) spectrum of the target with VLT/UVES, an echelle spectrograph operating in the near-UV to visible wavelengths (3200--6800 Å). We report several resolved $\Hi$ (H3--H6) and $\Hei$ emission lines ( $\lambda5875.6$ ) in the spectrum. Based on the asymmetric line profiles of $\Ha$ and $\Hb$ , 10 \% width of $\Ha$ ( $199\pm1$  $\kms$ ), tentative $\Hei$  $\lambda6678$ emission and indications of a disk from MIR excess, we confirm ongoing accretion at this object. Using the _Gaia_ update of the parallax, we revise its temperature to $1816\pm63$ K and radius to $1.5\pm0.1$ $\rj$ . Analysis of observed $\Hi$ profiles using 1D planet-surface shock model implies a pre-shock gas velocity of $v_0=120^{+80}_{-40}$  $\kms$ and a pre-shock density of $\log(n_0/\mathrm{cm}^{-3})=14^{+0}_{-5}$ . The pre-shock velocity points to a mass of $\Mp=6^{+8}_{-4} \mj$ for the target. Combining the $\Hi$ line luminosities and planetary $\Lline$ -- $\Lacc$ scaling relations, we derive a mass accretion rate of $\mdot$ $=1.4^{+2.8}_{-0.9}\times10^{-8}$  $\mjyr$ . The line-emitting area predicted from planet-surface shock model is very small ( $\sim0.03\%$ ), and points to a shock at the base of a magnetospherically induced funnel. The $\Ha$ profile exhibits much stronger flux than predicted by the model that best fits the rest of the $\Hi$ profiles, indicating that another mechanism than shock emission contributes to the $\Ha$ emission. Comparison of line fluxes and mass accretion rates from archival moderate-resolution SDSS spectra indicate variable accretion at 2MASS J11151597+1937266.

</div>

<div id="div_fig1">

<img src="tmp_2409.12187/./Hgamma34.png" alt="Fig2.1" width="33%"/><img src="tmp_2409.12187/./Hgamma35.png" alt="Fig2.2" width="33%"/><img src="tmp_2409.12187/./H_delta.png" alt="Fig2.3" width="33%"/>

**Figure 2. -** Line profiles of (upper panel) H$\gamma$ in order 34, (middle panel) H$\gamma$ in order 35, and (lower panel) H$\delta$ in order 28 detected in the UVES spectrum for 2M1115 in the Blue arm. The flux uncertainties are shown as vertical error bars and represent the weighted standard deviation among the four observations. Velocity is set with respect to the rest-frame wavelength (in air) of the respective lines. (*fig2*)

</div>
<div id="div_fig2">

<img src="tmp_2409.12187/./SED_fit_new.png" alt="Fig5" width="100%"/>

**Figure 5. -** Least-$\chi^2$ fits to the SED of 2M1115 with DRIFT-PHOENIX, BT-Settl-AGSS2009 and AMES-Dusty atmospheric models. The overall best fit with the minimum $\chi^2$
    corresponds to DRIFT-PHOENIX model at $\Teff=1800\pm50$ K, log(\textsl{g})$=4.0\pm0.25$ and metallicity$=0.3\pm0.15$. The SDSS DR12 spectrum for 2M1115 is also shown, along with its flux uncertainty (grey shaded region). Note that the flux in the WISE $W4$ band is a 2$\sigma$ upper limit, denoted by the red downward-pointing arrow, and is not included in the fitting process. The unWISE catalogue (\citealp{lang16}; see \url{https://unwise.me/photsearch/}) however reports a W4 detection consistent with this upper limit. (*apfig5*)

</div>
<div id="div_fig3">

<img src="tmp_2409.12187/./Halpha.png" alt="Fig1.1" width="50%"/><img src="tmp_2409.12187/./Hbeta.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** Line profiles of $\Ha$ in the RedU arm (top panel), and $\Hb$ in the RedL arm (bottom panel) of the UVES spectrum of 2M1115. NC, BC indicate narrow component and broad component, respectively, of the total Gaussian fit to the profile. The flux uncertainties are shown as vertical error bars and represent the weighted standard deviation among the four observations. Velocity is set with respect to the rest-frame wavelength (in air) of the respective lines. (*fig1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.12187"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

185  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
